<a href="https://colab.research.google.com/github/ali1810/Solubility_predition_feature/blob/main/Solubility_delaney_datset_rf_xg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
sol = pd.read_csv('/content/drive/MyDrive/delaney.csv')
sol

,Compound ID,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),SMILES
0,"1,1,1,2-Tetrachloroethane",-2.180,-2.794,ClCC(Cl)(Cl)Cl
1,"1,1,1-Trichloroethane",-2.000,-2.232,CC(Cl)(Cl)Cl
2,"1,1,2,2-Tetrachloroethane",-1.740,-2.549,ClC(Cl)C(Cl)Cl
3,"1,1,2-Trichloroethane",-1.480,-1.961,ClCC(Cl)Cl
4,"1,1,2-Trichlorotrifluoroethane",-3.040,-3.077,FC(F)(Cl)C(F)(Cl)Cl
...,...,...,...,...
1139,vamidothion,1.144,-1.446,CNC(=O)C(C)SCCSP(=O)(OC)(OC)
1140,Vinclozolin,-4.925,-4.377,CC1(OC(=O)N(C1=O)c2cc(Cl)cc(Cl)c2)C=C
1141,Warfarin,-3.893,-3.913,CC(=O)CC(c1ccccc1)c3c(O)c2ccccc2oc3=O
1142,Xipamide,-3.790,-3.642,Cc1cccc(C)c1NC(=O)c2cc(c(Cl)cc2O)S(N)(=O)=O


In [ ]:
sol["smiles_length"]= sol["SMILES"].apply(len)

In [ ]:
max_lenghth=max(sol["smiles_length"])
max_lenghth

98

In [ ]:
#smiles_length = dataset["SMILES"],key =len
#print(smiles_length)
sol  = sol[sol['smiles_length'] < 100]

In [ ]:
sol.SMILES

0                                    ClCC(Cl)(Cl)Cl
1                                      CC(Cl)(Cl)Cl
2                                    ClC(Cl)C(Cl)Cl
3                                        ClCC(Cl)Cl
4                               FC(F)(Cl)C(F)(Cl)Cl
                           ...                     
1139                   CNC(=O)C(C)SCCSP(=O)(OC)(OC)
1140          CC1(OC(=O)N(C1=O)c2cc(Cl)cc(Cl)c2)C=C
1141         CC(=O)CC(c1ccccc1)c3c(O)c2ccccc2oc3=O 
1142    Cc1cccc(C)c1NC(=O)c2cc(c(Cl)cc2O)S(N)(=O)=O
1143                         CNC(=O)Oc1cc(C)cc(C)c1
Name: SMILES, Length: 1144, dtype: object

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:40
🔁 Restarting kernel...


In [ ]:
### Installing RDKIT 

!mamba install -c conda-forge rdkit


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.8.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

conda-forge/linux-64     Using cache
conda-forge/noarch       Using cache
pkgs/main/noarc

**Convert SMILES in t molecule **

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

Calculate molecular descriptors
To predict LogS (log of the aqueous solubility), the study by Delaney makes use of 4 molecular descriptors:

cLogP (Octanol-water partition coefficient)
MW (Molecular weight)
RB (Number of rotatable bonds)
AP (Aromatic proportion = number of aromatic atoms / total number of heavy atoms)
Unfortunately, rdkit readily computes the first 3. As for the AP descriptor, we will calculate this by manually computing the ratio of the number of aromatic atoms to the total number of heavy atoms which rdkit can compute. 

####  Calculate LogP, MW and RB

In [ ]:
import numpy as np
from rdkit.Chem import Descriptors
from rdkit.Chem import Lipinski
from rdkit.Chem import Crippen

In [ ]:
#desc_Ringcount        =   Descriptors.desc_Ringcount(mol)


In [ ]:
### Write a function to calculate these values....

def getAromaticProportion(m):
    aromatic_list = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
    aromatic = 0
    for i in aromatic_list:
        if i:
            aromatic += 1
    heavy_atom = Lipinski.HeavyAtomCount(m)
    return aromatic / heavy_atom


def generate(smiles):
    moldata = []
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData = np.arange(1, 1)
    i = 0
    for mol in moldata:

        desc_MolLogP = Crippen.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Lipinski.NumRotatableBonds(mol)
        desc_AromaticProportion = getAromaticProportion(mol)
        desc_Ringcount        =   Descriptors.RingCount(mol)
        desc_TPSA = Descriptors.TPSA(mol)
        desc_Hdonrs=Lipinski.NumHDonors(mol)
        desc_SaturatedRings = Lipinski.NumSaturatedRings(mol)   
        desc_AliphaticRings = Lipinski.NumAliphaticRings(mol) 
        desc_HAcceptors = Lipinski.NumHAcceptors(mol)
        desc_Heteroatoms = Lipinski.NumHeteroatoms(mol)
        #desc_molMR=Descriptors.MolMR(mol)
        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds,
                        desc_AromaticProportion,desc_Ringcount,desc_TPSA,desc_Hdonrs,desc_SaturatedRings,desc_AliphaticRings,desc_HAcceptors,desc_Heteroatoms])

        if i == 0:
            baseData = row
        else:
            baseData = np.vstack([baseData, row])
        i = i + 1

    columnNames = ["MolLogP", "MolWt", "NumRotatableBonds", "AromaticProportion","Ring_Count","TPSA","H_donors","Saturated_Rings","AliphaticRings","H_Acceptors","Heteroatoms"]
    descriptors = pd.DataFrame(data=baseData, columns=columnNames)

    return descriptors


In [ ]:
#### Get new dataframe 
df = generate(sol.SMILES)
#print(sol.SMILES)
print(df)
#df.head(100)

      MolLogP    MolWt  ...  H_Acceptors  Heteroatoms
0     2.59540  167.850  ...          0.0          4.0
1     2.37650  133.405  ...          0.0          3.0
2     2.59380  167.850  ...          0.0          4.0
3     2.02890  133.405  ...          0.0          3.0
4     2.91890  187.375  ...          0.0          6.0
...       ...      ...  ...          ...          ...
1139  1.98820  287.343  ...          6.0          8.0
1140  3.42130  286.114  ...          3.0          6.0
1141  3.60960  308.333  ...          4.0          4.0
1142  2.56214  354.815  ...          4.0          8.0
1143  2.02164  179.219  ...          2.0          3.0

[1144 rows x 11 columns]


In [ ]:
print(type(sol.SMILES[0]))

<class 'str'>


3.2.2. Aromatic proportion

3.2.1.1. Number of aromatic atoms

Here, we will create a custom function to calculate the Number of aromatic atoms. With this descriptor we can use it to subsequently calculate the AP descriptor.

Computing for a single molecule.

In [ ]:
Y = sol.iloc[:,1]
Y
X=df


In [ ]:
####  data split ...

from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, Y, test_size=0.05, random_state=42)

In [ ]:

X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

(1086, 11)
(58, 11)
(58,)
(1086,)


In [ ]:
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)




LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
model_svr=SVR(kernel='rbf')
model_svr.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
model_rf = RandomForestRegressor(n_estimators = 10000, random_state = 0)
model_rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10000, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [ ]:
Y_pred_test_rf = model_rf.predict(X_test)

In [ ]:
from xgboost import XGBRegressor
model_xgr = XGBRegressor(n_estimators = 10000, random_state = 0)
model_xgr.fit(X_train, y_train)

[12:57:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=10000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
Y_pred_test_xgr = model_xgr.predict(X_test)

In [ ]:
Y_pred_test_rf = model_rf.predict(X_test)
#res = "\n".join("{} {}".format(x, y) for x, y in zip(y_test,Y_pred_test_rf ))
#print(res)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, Y_pred_test_rf)

0.9269270505484866

In [ ]:
import pickle
#### Save the model :::::
filename = 'finalized_model.rf'
pickle.dump(model_rf, open('/content/drive/MyDrive/finalized_model.rf', 'wb'))
 



In [ ]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open('/content/drive/MyDrive/finalized_model.rf', 'rb'))
Y_pred_test_rf = loaded_model.predict(X_test)
result = r2_score(y_test, Y_pred_test_rf)
print(result)

0.9269270505484866


In [ ]:
elem=input("Enter smiles get solubility ")
#elem='CCc1ccccc1C'
#moldata = []
#for elem in smiles:
mol = Chem.MolFromSmiles(elem)
#moldata.append(mol)

#    baseData = np.arange(1, 1)
#    i = 0
#    for mol in moldata:

desc_MolLogP = Crippen.MolLogP(mol)
desc_MolWt = Descriptors.MolWt(mol)
desc_NumRotatableBonds = Lipinski.NumRotatableBonds(mol)
desc_AromaticProportion = getAromaticProportion(mol)
desc_Ringcount        =   Descriptors.RingCount(mol)
desc_TPSA = Descriptors.TPSA(mol)
desc_Hdonrs=Lipinski.NumHDonors(mol)
desc_SaturatedRings = Lipinski.NumSaturatedRings(mol)   
desc_AliphaticRings = Lipinski.NumAliphaticRings(mol) 
desc_HAcceptors = Lipinski.NumHAcceptors(mol)
desc_Heteroatoms = Lipinski.NumHeteroatoms(mol)
#desc_molMR=Descriptors.MolMR(mol)
row = np.array([desc_MolLogP,desc_MolWt,desc_NumRotatableBonds,desc_AromaticProportion,desc_Ringcount,desc_TPSA,desc_Hdonrs,desc_SaturatedRings,desc_AliphaticRings,desc_HAcceptors,desc_Heteroatoms])
                     #   
                     #   
                     #   )

        #if i == 0:
baseData = row
        #else:
baseData = np.vstack([row])
#i = i + 1

columnNames = ["MolLogP", "MolWt", "NumRotatableBonds", "AromaticProportion","Ring_Count","TPSA","H_donors","Saturated_Rings","AliphaticRings","H_Acceptors","Heteroatoms"]
descriptors = pd.DataFrame(data=baseData, columns=columnNames)
descriptors =np.array(descriptors) 
preds=loaded_model.predict(descriptors)
#predict_test_input = smiles_to_onehots(['elem'], charset, max_smiles_chars)
#loaded_model1 = load_model("/content/drive/MyDrive/solubility_model_8858.hdf5")
#solubility_prediction = loaded_model.predict(predict_test_input)
print(preds)
#print(solubility_prediction)
#print(type(descriptors))    


Enter smiles get solubility Fc1ccc(cc1)C(=O)c2ccc(F)cc2
[-3.48531444]
